In [2]:
import pandas as pd
import yfinance as yf
import datetime as dt


In [17]:
symbol = "CHFUSD=X"
interval = '1mo'
period = '3y'

In [18]:
df_weekly = yf.download(symbol, interval=interval, period=period)
df_weekly = df_weekly.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed



In [19]:
df_weekly.head()

Price,Close,High,Low,Open,Volume
Ticker,CHFUSD=X,CHFUSD=X,CHFUSD=X,CHFUSD=X,CHFUSD=X
Date,,,,,
2025-07-01,1.248642,1.270002,1.240649,1.261479,0
2025-06-01,1.252740,1.259763,1.212386,1.216848,0
2025-05-01,1.216307,1.221449,1.180178,1.210712,0
2025-04-01,1.214447,1.239587,1.130199,1.131452,0
2025-03-01,1.135267,1.142074,1.107265,1.107265,0


In [14]:

symbol = "CHFUSD=X"
df = yf.download(symbol, period="3y", interval="1d")
df.index = pd.to_datetime(df.index)

# Flatten columns
df.columns = df.columns.get_level_values(0)

# Resample by week ending Fri
weekly = df.resample("W-FRI").agg(
    open=("Open", "first"),
    high=("High", "max"),
    low=("Low", "min"),
    close=("Close", "last"),
    volume=("Volume", "sum")
)

print(weekly.tail())

[*********************100%***********************]  1 of 1 completed

                open      high       low     close  volume
Date                                                      
2025-06-20  1.231200  1.236400  1.217374  1.224905       0
2025-06-27  1.222868  1.256266  1.220465  1.248611       0
2025-07-04  1.252740  1.270002  1.251048  1.259287       0
2025-07-11  1.259382  1.262945  1.251016  1.255146       0
2025-07-18  1.255036  1.257624  1.240649  1.248642       0


In [16]:
symbol = "CHFUSD=X"
df = yf.download(symbol, period="3y", interval="1d")
df.index = pd.to_datetime(df.index)

today = pd.Timestamp(dt.datetime.now().date())
weekday = today.weekday()

if weekday == 0 and today in df.index:
    price = df.loc[today, "Open"]
    source = f"Monday open ({today.date()})"
else:
    last_friday = today - pd.offsets.Week(weekday=4)
    if last_friday not in df.index:
        last_friday = df.index[df.index <= last_friday].max()
    price = df.loc[last_friday, "Close"]
    source = f"Friday close ({last_friday.date()})"

print(f"{source}: {price}")

[*********************100%***********************]  1 of 1 completed

Friday close (2025-07-18): Ticker
CHFUSD=X    1.248642
Name: 2025-07-18 00:00:00, dtype: float64


In [5]:
df3 = yf.download("CHFUSD=X", period="3mo", interval="1d")
df3.columns = df3.columns.get_level_values(0)
df3.index = pd.to_datetime(df3.index)

weekly = df3.resample("W-MON").agg({
    "Open": "first",
    "High": "max",
    "Low": "min",
    "Close": "last",
    "Volume": "sum"
})

print(weekly.tail())

[*********************100%***********************]  1 of 1 completed

Price           Open      High       Low     Close  Volume
Date                                                      
2025-06-23  1.228939  1.231830  1.217374  1.222868       0
2025-06-30  1.230648  1.259763  1.230058  1.252740       0
2025-07-07  1.261479  1.270002  1.252223  1.259382       0
2025-07-14  1.254453  1.262945  1.251016  1.255036       0
2025-07-21  1.253337  1.257624  1.240649  1.249453       0


In [10]:
import yfinance as yf
from datetime import datetime
import pytz

def fetch_night_close(symbol):
    t = yf.Ticker(symbol)
    # Method A: quick last close
    last_close = t.fast_info.previous_close

    # Method B: confirm with history
    data = t.history(period="1d", interval="1d")
    close_hist = data["Close"].iloc[0]

    return last_close, close_hist

if __name__ == "__main__":
    now = datetime.now(pytz.utc)
    print("Night fetch at:", now)
    closeA, closeB = fetch_night_close("AAPL")
    print("fast_info:", closeA, "| history:", closeB)

Night fetch at: 2025-07-22 07:35:20.195712+00:00
fast_info: 211.26 | history: 212.47999572753906
fast_info: 211.26 | history: 212.47999572753906
